In [1]:
import sys, csv
from modules.wordTokenizer import WordTokenizer
from modules.charTokenizer import CharTokenizer
from modules.CNNLSTMCRFNetwork import CNNLSTMCRFNetwork
import numpy as np

Using TensorFlow backend.
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type

In [2]:
MAX_TEXTS = 1000000
#Read product file
texts, tags, brands = [], [], []
with open("data/train_products.csv", 'r') as f:
    reader = csv.DictReader(f, fieldnames=["title","brand","tags"])
    count = 0
    for row in reader:
        #print(row)
        count += 1
        text, tag_set = row['title'], row['tags'].split(' ')[:-1]
        texts.append(text)
        tags.append(tag_set)
        brands.append(row['brand'])
        if count >= MAX_TEXTS:
            break

In [3]:
#Create training examples
numSentences = len(texts)
np.random.seed(0)
training_examples = np.random.binomial(1, 0.7, numSentences)

In [4]:
#Separate data in training and test
trainData = []
testData  = []

nerLabels_train =[]
nerLabels_test = []


for example in range(numSentences):
    if training_examples[example] == 1:
        trainData.append(texts[example])
        nerLabels_train.append(tags[example])
    else:
        testData.append(texts[example])
        nerLabels_test.append(tags[example])

In [5]:
#create subset of train data
numTrainSentences = 1000

trainData_tiny       = trainData[0:numTrainSentences]
nerLabels_train_tiny = nerLabels_train[0:numTrainSentences]

In [6]:
#Initiate network
network = CNNLSTMCRFNetwork()

In [7]:
#Create tokenized sentences
wordTokenizer = WordTokenizer(30,'models/word_tokenizer')
wordTokenizer.load()
data = wordTokenizer.tokenize(trainData_tiny)
testDataTokenized = wordTokenizer.tokenize(testData)

In [8]:
charTokenizer = CharTokenizer(10,'models/word_tokenizer',30)
charTokenizer.load()
charData = charTokenizer.tokenize(trainData_tiny)
testCharData = charTokenizer.tokenize(testData)

In [9]:
#Generate pad labels for network
labels = network.get_labels(nerLabels_train_tiny,wordTokenizer)
testLabels = network.get_labels(nerLabels_test,wordTokenizer)

Getting labels...
Getting labels...


In [10]:
network.compile(wordTokenizer,charTokenizer)

Loading GloVe embedding...


W0805 19:19:52.864765 139995219748672 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 19:19:52.889798 139995219748672 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 19:19:52.890931 139995219748672 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 19:19:52.900835 139995219748672 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final

Found 855381 word vectors.
Creating embedding layer...
Creating network...
Start word embeddings


W0805 19:19:56.161739 139995219748672 deprecation.py:506] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Start char embeddings
Concat models


W0805 19:19:56.506765 139995219748672 deprecation.py:323] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Add crf layer


/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '
W0805 19:19:57.275405 139995219748672 deprecation_wrapper.py:119] From /home/felipe/.local/share/virtualenvs/w266_final_project-K7oLnwuw/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Compiling network...


In [11]:
network.train([data,charData], labels, epochs=8, validation_split=0.5, batch_size=32)

Training...
(1000, 30) [[[1.040e+02 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [9.880e+02 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [3.030e+02 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  ...
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]

 [[7.320e+02 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [1.072e+03 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [7.630e+02 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  ...
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]]

 [[1.047e+03 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [5.760e+03 0.000e+00 0.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
  [7.000e

In [12]:
network.evaluate([testDataTokenized,testCharData],testLabels)

Evaluating...
18558/18558 [==============================] - 6s 318us/step
              precision    recall  f1-score   support

           O      0.981     0.994     0.988    532625
         B-B      0.760     0.545     0.635     18584
         I-B      0.839     0.534     0.652      5531

    accuracy                          0.975    556740
   macro avg      0.860     0.691     0.758    556740
weighted avg      0.972     0.975     0.973    556740

